In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from datetime import datetime
import os
import unidecode
from scipy import stats

In [2]:
#Funções para tratamento de registro com espaços desnecessário
def funcLimparEspacos(df,col):
    palavras = df[col]
    palavras = palavras.strip() 
    return palavras


def removerAcentuacaoCidade(df,col):
    cidade = unidecode.unidecode(df[col])
    return cidade

In [3]:
#Lendo arquivo CSV com registro dos acidentes no ano de 2016
dataset_2016 = pd.read_csv('csv/datatran2016.csv',encoding='ISO-8859-1',sep=";")

In [4]:
#Padronizando o dataframe com os dados relevantes para analise
dataset_2016 = dataset_2016.drop(dataset_2016.columns[[0,3,5,6,9,10,11,12,14,15,16,19,20,21,22,24]], axis=1)

In [5]:
#Filtrando os acidentes ocorridos no Estado de PE
dataset_2016 = dataset_2016[(dataset_2016.uf == "PE")]

In [6]:
#Padronizando o formato de data
dataset_2016.data_inversa=pd.to_datetime(dataset_2016.data_inversa).dt.strftime('%Y-%m-%d')
dataset_2016.rename(columns={'data_inversa':'Date'}, inplace=True)

In [7]:
#----Tratamento na Base - Limpeza no excesso de espaços nos resgistros de municipio e causa_acidente ----
dataset_2016['municipio'] = dataset_2016.apply(funcLimparEspacos,col = 3, axis=1)
dataset_2016['causa_acidente'] = dataset_2016.apply(funcLimparEspacos,col = 4, axis=1)

In [8]:
#Separacao da data
dataset_2016['year'] = pd.DatetimeIndex(dataset_2016['Date']).year
dataset_2016['month'] = pd.DatetimeIndex(dataset_2016['Date']).month
dataset_2016['day'] = pd.DatetimeIndex(dataset_2016['Date']).day

#Contador unitario para cada acidente
dataset_2016['acidente'] = 1

In [9]:
#Lendo arquivo CSV com registro de precipitação em PE
dataset_Precipitacao = pd.read_csv('csv/result_monthly_precipitation_sum.csv')

In [10]:
#Filtrando o Municipio de RECIFE do Dataframe de acidentes
dataset_2016_Recife = dataset_2016[(dataset_2016.municipio == "RECIFE")]

In [11]:
#Filtrando ano de 2016 da Base de Precipitação e o Municipio de Recife
dataset_Precipitacao_2016 = dataset_Precipitacao[(dataset_Precipitacao.year == 2016)]
dataset_Precipitacao_2016_Recife = dataset_Precipitacao_2016[(dataset_Precipitacao_2016.Estacao == "RECIFE")]
dataset_Precipitacao_2016_Recife.rename(columns={'Estacao':'municipio'}, inplace=True)

c:\users\carlos\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [15]:
#Filtrando os acidentes causados por Ingestão de Álcool
dataset_2016_Recife_Acidentes_alcool = dataset_2016_Recife[(dataset_2016_Recife.causa_acidente == "Ingestão de álcool")]

#Aplicando somatorio mensal dos acidentes causados por Ingestão de Álcool
dataset_2016_Recife_Acidentes_alcool_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2016_Recife_Acidentes_alcool.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
df_precipitacao_x_Acidentes_alcool_somados = pd.merge(dataset_Precipitacao_2016_Recife, dataset_2016_Recife_Acidentes_alcool_somados, on = ('municipio','month'))

In [16]:
#Coeficiente de Correlação Spearman 
df_precipitacao_x_Acidentes_alcool_somados['Precipitacao'].corr(df_precipitacao_x_Acidentes_alcool_somados['soma_acidentes_mes'], method='spearman')

-0.07824607964359516

In [18]:
#Filtrando os acidentes causados pela falta de atenção do condutor
dataset_2016_Recife_Acidentes_condutor_desatento = dataset_2016_Recife[(dataset_2016_Recife.causa_acidente == "Falta de atenção")]

#Aplicando somatorio mensal dos acidentes causados pela falta de atenção do condutor
dataset_2016_Recife_Acidentes_condutor_desatento_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2016_Recife_Acidentes_condutor_desatento.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
dataset_2016_Recife_Acidentes_condutor_desatento_somados = pd.merge(dataset_Precipitacao_2016_Recife, dataset_2016_Recife_Acidentes_condutor_desatento_somados, on = ('municipio','month'))

In [19]:
#Coeficiente de Correlação Spearman 
dataset_2016_Recife_Acidentes_condutor_desatento_somados['Precipitacao'].corr(dataset_2016_Recife_Acidentes_condutor_desatento_somados['soma_acidentes_mes'], method='spearman')

-0.03873335483621993

In [21]:
#Filtrando os acidentes causados pela falta de distancia de segura entre os veiculos
dataset_2016_Recife_Acidentes_condutor_distancia = dataset_2016_Recife[(dataset_2016_Recife.causa_acidente == "Não guardar distância de segurança")]

#Aplicando somatorio mensal dos acidentes causados pela falta de distancia de segura entre os veiculos
dataset_2016_Recife_Acidentes_condutor_distancia_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2016_Recife_Acidentes_condutor_distancia.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
dataset_2016_Recife_Acidentes_condutor_distancia_somados = pd.merge(dataset_Precipitacao_2016_Recife, dataset_2016_Recife_Acidentes_condutor_distancia_somados, on = ('municipio','month'))


In [22]:
#Coeficiente de Correlação Spearman 
dataset_2016_Recife_Acidentes_condutor_distancia_somados['Precipitacao'].corr(dataset_2016_Recife_Acidentes_condutor_distancia_somados['soma_acidentes_mes'], method='spearman')

-0.2619670089686946

In [24]:
#Filtrando os acidentes causados por defeitos na via
dataset_2016_Recife_Acidentes_defeito_via = dataset_2016_Recife[(dataset_2016_Recife.causa_acidente == "Defeito na via")]

#Aplicando somatorio mensal dos acidentes causados por defeitos na via
dataset_2016_Recife_Acidentes_defeito_via_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2016_Recife_Acidentes_defeito_via.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
dataset_2016_Recife_Acidentes_defeito_via_somados = pd.merge(dataset_Precipitacao_2016_Recife, dataset_2016_Recife_Acidentes_defeito_via_somados, on = ('municipio','month'))

In [25]:
#Coeficiente de Correlação Spearman 
dataset_2016_Recife_Acidentes_defeito_via_somados['Precipitacao'].corr(dataset_2016_Recife_Acidentes_defeito_via_somados['soma_acidentes_mes'], method='spearman')

0.13762047064079508

In [27]:
#Filtrando os acidentes causados por animais na via
dataset_2016_Recife_Acidentes_animais_pista = dataset_2016_Recife[(dataset_2016_Recife.causa_acidente == "Animais na Pista")]

#Aplicando somatorio mensal dos acidentes causados por defeitos na via
dataset_2016_Recife_Acidentes_animais_pista_somados = pd.DataFrame({'soma_acidentes_mes' : dataset_2016_Recife_Acidentes_animais_pista.groupby(["municipio", "month"])["acidente"].sum()}).reset_index()
dataset_2016_Recife_Acidentes_animais_pista_somados = pd.merge(dataset_Precipitacao_2016_Recife, dataset_2016_Recife_Acidentes_animais_pista_somados, on = ('municipio','month'))

In [28]:
#Coeficiente de Correlação Spearman 
dataset_2016_Recife_Acidentes_animais_pista_somados['Precipitacao'].corr(dataset_2016_Recife_Acidentes_animais_pista_somados['soma_acidentes_mes'], method='spearman')

-0.50709255283711